In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Insert rows from temp dependency view into ETL_LOG_DEPENDENCIES
INSERT_ROW = ", ".join(
    spark.table(f"{catalog_name}.{metadata_schema}.ETL_LOG_DEPENDENCIES").columns
)

print((f"""
    INSERT INTO {catalog_name}.{metadata_schema}.ETL_LOG_DEPENDENCIES ({INSERT_ROW})
    SELECT {INSERT_ROW} FROM {target_table}_DEPENDENCIES
"""))

spark.sql(f"""
    INSERT INTO {catalog_name}.{metadata_schema}.ETL_LOG_DEPENDENCIES ({INSERT_ROW})
    SELECT {INSERT_ROW} FROM {target_table}_DEPENDENCIES
""")


In [0]:
spark.read.table("udp_wcm_dev.udp_Wcm_metadata_dev.ETL_LOG_DEPENDENCIES")

In [0]:
max_proc_date = spark.sql(
    f"select max(proc_date) from {catalog_name}.{metadata_schema}.ETL_LOG_DEPENDENCIES"
).head()[0]
curr_rec_etl_dep_df = spark.sql(
    f"select * from {catalog_name}.{metadata_schema}.ETL_LOG_DEPENDENCIES where proc_date = '{max_proc_date}'"
)
curr_rec_etl_dep_df.display()

In [0]:
# Insert rows into ETL_DELTA_TABLE
spark.sql(
    f"""
    INSERT INTO {catalog_name}.{metadata_schema}.ETL_DELTA_TABLE
    SELECT 
        current_timestamp() + INTERVAL 7 HOURS AS PROC_DATE
        , "{CALDAY_IN_STR}" AS CALDAY_IN_STR
        , "{target_dataset}" AS DATASET_AFFECT
        , "{target_table}" AS TABLE_AFFECT
        , "{field_calday}" FIELD_CALDAY
        , "{field_id}" FIELD_ID
        , 1 STATUS
        , CAST("{START_DATE_D}" AS DATE) AS START_DATE_D
        , CAST("{END_DATE_D}" AS DATE) AS END_DATE_D
        , "{TABLE_ETL}" AS TABLE_ETL
"""
)

In [0]:
max_proc_date = spark.sql(
    f"select max(proc_date) from {catalog_name}.{metadata_schema}.ETL_DELTA_TABLE"
).head()[0]
curr_rec_etl_del_df = spark.sql(
    f"select * from {catalog_name}.{metadata_schema}.ETL_DELTA_TABLE where proc_date = '{max_proc_date}'"
)
curr_rec_etl_del_df.display()